In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\CaptionAI\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\CaptionAI'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    dataset_link: str
    local_data_file: Path
    unzip_dir: Path

Configuration

In [6]:
from CaptionAI.constants import *
from CaptionAI.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            dataset_link = config.dataset_link,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

Components

In [ ]:
import time
import os
import requests
from kagglehub import dataset_download
from CaptionAI import logger

c:\Users\rahul\Desktop\Project\CaptionAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if os.path.exists(self.config.local_data_file):
            logger.info("Dataset already exists.")
            return

        retries = 3
        for attempt in range(retries):
            try:
                logger.info(f"Attempt {attempt + 1} to download the dataset...")
                
                data_path = dataset_download(self.config.dataset_link)
                logger.info("Dataset has been downloaded successfully.")

                os.rename(src=data_path, dst=self.config.local_data_file)
                logger.info(f"Dataset has been moved to: {self.config.local_data_file}")
                return  
            except requests.exceptions.Timeout:
                logger.warning(f"Timeout occurred during download attempt {attempt + 1}. Retrying...")
            except Exception as e:
                logger.error(f"Error occurred during download: {e}")
                if attempt == retries - 1:
                    raise  
            time.sleep(2 ** attempt)  

Pipeline

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-12-04 09:27:11,153: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 09:27:11,154: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 09:27:11,155: INFO: common: created directory at: artifacts]
[2024-12-04 09:27:11,157: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-04 09:27:11,157: INFO: 3650788447: Dataset already exists.]
